In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Concatenate
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model

import os
import shutil
import glob
#from keras import backend as K
#K.set_image_dim_ordering('th')

import numpy as np
import pandas as pd
#import h5py

import matplotlib.pyplot as plt

## 1. Препроцессинг изображений

### 1.1 Очистка папок

In [2]:
def clear_folder(mydir):
    filelist = [f for f in os.listdir(mydir)]
    for f in filelist:
        os.remove(os.path.join(mydir, f))

In [3]:
for i in os.listdir('data/train_a'):
    clear_folder(f'data/train_a/{i}')
    
for i in os.listdir('data/test_a'):
    clear_folder(f'data/test_a/{i}')

for i in os.listdir('data/train_c'):
    clear_folder(f'data/train_c/{i}')
    
for i in os.listdir('data/test_c'):
    clear_folder(f'data/test_c/{i}')

### 1.2 Выбор изображений и заполнение папок

In [4]:
def fill_folder_by_class(name):
    data_names_a = list(glob.glob(f'data/{name}/*Plate_?*_?*a.tif'))
    data_names_c = list(glob.glob(f'data/{name}/*Plate_?*_?*c.tif'))
    f_1 = open(f'data/{name}/different_angle.txt', 'rt')
    f_2 = open(f'data/{name}/miss_view.txt', 'rt')
    diff_angle = f_1.readlines()
    miss_view = f_2.readlines()
    for i, pic in enumerate(diff_angle):
        diff_angle[i] = f'data/{name}\\{pic.rstrip()}.tif'#обрезаем символы на конце строки '\n'
    for i, pic in enumerate(miss_view):
        miss_view[i] = f'data/{name}\\{pic.rstrip()}.tif'

    for pic in diff_angle:
        if pic in data_names_a:
            data_names_a.remove(pic)
            if pic[:-5]+'c.tif' in data_names_c:
                data_names_c.remove(pic[:-5]+'c.tif')
    for pic in diff_angle:
        if pic in data_names_c:
            data_names_c.remove(pic)

    for pic in miss_view:
        if (pic[-5:-4] == 'c')==True and (pic[:-5]+'a.tif' in diff_angle)==False and (pic[:-5]+'a.tif' in miss_view)==False:
            data_names_a.remove(pic[:-5]+'a.tif')
        if pic[-5:-4] == 'a' and (pic[:-5]+'c.tif' in diff_angle)==False and (pic[:-5]+'c.tif' in miss_view)==False:
            data_names_c.remove(pic[:-5]+'c.tif')

    f_1.close()
    f_2.close()
    
    class_name = name.partition('_')[-1].partition('_')[0][0:3]
    if not os.path.exists(f'data/train_a/class_{class_name}'):
        os.makedirs(f'data/train_a/class_{class_name}')
    if not os.path.exists(f'data/test_a/class_{class_name}'):
        os.makedirs(f'data/test_a/class_{class_name}')
    if not os.path.exists(f'data/train_c/class_{class_name}'):
        os.makedirs(f'data/train_c/class_{class_name}')
    if not os.path.exists(f'data/test_c/class_{class_name}'):
        os.makedirs(f'data/test_c/class_{class_name}')

    for i, n in enumerate(range(len(data_names_a))):
        if i <= int(len(data_names_a)*0.75):
            shutil.copy(data_names_a[n], f'data/train_a/class_{class_name}')
        else:
            shutil.copy(data_names_a[n], f'data/test_a/class_{class_name}')
    for i, n in enumerate(range(len(data_names_c))):
        if i <= int(len(data_names_c)*0.75):
            shutil.copy(data_names_c[n], f'data/train_c/class_{class_name}')
        else:
            shutil.copy(data_names_c[n], f'data/test_c/class_{class_name}')

In [5]:
fill_folder_by_class('Aldanella_attleborensis')
fill_folder_by_class('Aldanella_sibirica_sp_nov')

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator_a = train_datagen.flow_from_directory('data/train_a/',
                                        target_size=(150, 150),
                                        batch_size=100,
                                        color_mode='rgb',
                                        class_mode='binary',
                                        shuffle=True)
train_generator_c = train_datagen.flow_from_directory('data/train_c/',
                                        target_size=(150, 150),
                                        batch_size=100,
                                        color_mode='rgb',
                                        class_mode='binary',
                                        shuffle=True)

validation_generator_a = test_datagen.flow_from_directory('data/test_a/',
                                              target_size=(150, 150),
                                              batch_size=100,
                                              color_mode='rgb',
                                              class_mode='binary',
                                              shuffle=True)
validation_generator_c = test_datagen.flow_from_directory('data/test_c/',
                                              target_size=(150, 150),
                                              batch_size=100,
                                              color_mode='rgb',
                                              class_mode='binary',
                                              shuffle=True)

'''
pred_generator=test_datagen.flow_from_directory('data/img_val/',
                                                     target_size=(150,150),
                                                     batch_size=100,
                                                     class_mode='binary')
'''

Found 59 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


"\npred_generator=test_datagen.flow_from_directory('data/img_val/',\n                                                     target_size=(150,150),\n                                                     batch_size=100,\n                                                     class_mode='binary')\n"

## 2. Создание модели

In [7]:
cnn_model_a = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 3)))

In [8]:
cnn_model_c = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 3)))

In [9]:
X_train_a, y_train_a = train_generator_a.next()
X_train_c, y_train_c = train_generator_c.next()

X_validation_a, y_validation_a = validation_generator_a.next()
X_validation_c, y_validation_c = validation_generator_c.next()

In [68]:
combinedInput = Concatenate()([cnn_model_a.output, cnn_model_c.output])
x = Dense(128, activation='relu')(combinedInput)
x = Dropout(0.5, name='dropout_c_one')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5, name='dropout_c_two')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
top = Dense(1, activation='sigmoid')(x)
model = Model(inputs=[cnn_model_a.input, cnn_model_c.input], outputs=top)

In [69]:
#plot_model(model, to_file='model_exp3_plot.png', show_shapes=True, show_layer_names=True)

In [70]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=1e-4, momentum=0.9),
                #optimizer='adam',
              #optimizer=Adam(learning_rate=1e-4, decay=1e-4 / 200),
              metrics=['accuracy'])

In [71]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [74]:
def num_files(path):
    num_files = len([f for f in os.listdir(path)])
    return num_files

In [75]:
train_labels_a = []
test_labels_a = []
filelist = [f for f in os.listdir('data/train_a')]
for i, f in enumerate(filelist):
    train_labels_a.extend([i]*num_files(f'data/train_a/{f}'))

filelist = [f for f in os.listdir('data/test_a')]
for i, f in enumerate(filelist):
    test_labels_a.extend([i]*num_files(f'data/test_a/{f}'))

train_labels_a = np.array(train_labels_a)
test_labels_a = np.array(test_labels_a)

In [76]:
model.fit([X_train_a, X_train_c], train_labels_a,
            validation_data=([X_validation_a, X_validation_c], y_validation_a),
            epochs=200, batch_size=10,
            callbacks=[early_stopping])

Epoch 1/200
6/6 [==============================] - 10s 422ms/step - loss: 1.1082 - accuracy: 0.4407 - val_loss: 0.9373 - val_accuracy: 0.3333
Epoch 2/200
6/6 [==============================] - 1s 121ms/step - loss: 0.8704 - accuracy: 0.4576 - val_loss: 0.8924 - val_accuracy: 0.3333
Epoch 3/200
6/6 [==============================] - 1s 121ms/step - loss: 0.9860 - accuracy: 0.3729 - val_loss: 0.8231 - val_accuracy: 0.3889
Epoch 4/200
6/6 [==============================] - 1s 123ms/step - loss: 0.9650 - accuracy: 0.4237 - val_loss: 0.7432 - val_accuracy: 0.3333
Epoch 5/200
6/6 [==============================] - 1s 135ms/step - loss: 0.9102 - accuracy: 0.4407 - val_loss: 0.6827 - val_accuracy: 0.3889
Epoch 6/200
6/6 [==============================] - 1s 121ms/step - loss: 0.6858 - accuracy: 0.6441 - val_loss: 0.6557 - val_accuracy: 0.7222
Epoch 7/200
6/6 [==============================] - 1s 121ms/step - loss: 0.6482 - accuracy: 0.6780 - val_loss: 0.6219 - val_accuracy: 0.7222
Epoch 8/200


In [77]:
result = model.evaluate([X_validation_a, X_validation_c], y_validation_a)
dict(zip(model.metrics_names, result))

1/1 [==============================] - 0s 215ms/step - loss: 0.6086 - accuracy: 0.7222


{'loss': 0.6086128950119019, 'accuracy': 0.7222222089767456}